In [3]:
import pdfplumber
import os
import umap
import re
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hdbscan
from nltk.tokenize import sent_tokenize
import nltk.data
import yake
import string

In [99]:
DB = pd.read_csv('dataset.csv')

data = DB[DB.text.str.len()>30]

In [100]:

data = data[['text', 'name','year']]
data

,text,name,year
0,sustainability design report sustainability im...,unipolsai,2020
1,sustainability summary measuring results obta...,unipolsai,2020
2,sustainability report complemented group docum...,unipolsai,2020
3,draft financial statements sustainability repo...,unipolsai,2020
4,addition several financial services sector dis...,unipolsai,2020
...,...,...,...
60999,naples houses martyrdom saint last documented...,intesa,2017
61000,over naples via toledo naples via toledo rain ...,intesa,2017
61001,artist particularly renowned famous streets si...,intesa,2017
61002,picture part permanent collection display muse...,intesa,2017


In [101]:
data = data.reset_index(drop=True)
data


,text,name,year
0,sustainability design report sustainability im...,unipolsai,2020
1,sustainability summary measuring results obta...,unipolsai,2020
2,sustainability report complemented group docum...,unipolsai,2020
3,draft financial statements sustainability repo...,unipolsai,2020
4,addition several financial services sector dis...,unipolsai,2020
...,...,...,...
57953,naples houses martyrdom saint last documented...,intesa,2017
57954,over naples via toledo naples via toledo rain ...,intesa,2017
57955,artist particularly renowned famous streets si...,intesa,2017
57956,picture part permanent collection display muse...,intesa,2017


In [102]:

model = SentenceTransformer('all-MiniLM-L6-v2')


emb = model.encode(data['text'],show_progress_bar=True)

Batches:   0%|          | 0/1812 [00:00<?, ?it/s]

In [103]:
umap_embeddings = umap.UMAP(n_neighbors=8, 
                            n_components=5, 
                            metric='cosine').fit_transform(emb)
            
cluster = hdbscan.HDBSCAN(min_cluster_size=8,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)
            
            # Prepare data
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(emb)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Avoid using `tokenizers` before the fork if possible
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
	- Explicitly set the environment variable T

In [104]:
df = pd.DataFrame(umap_data, columns=['x', 'y'])
df

,x,y
0,7.532803,6.918570
1,6.878398,2.374060
2,6.626009,2.709674
3,6.647938,2.830496
4,6.641014,2.785725
...,...,...
57953,9.534426,6.548452
57954,9.531287,6.496984
57955,9.533686,6.537910
57956,9.526855,6.433837


In [105]:

data['x'], data['y'] = df['x'], df['y']
data['labels'] = cluster.labels_




In [106]:
data = data[data['labels']!=-1]
data

,text,name,year,x,y,labels
0,sustainability design report sustainability im...,unipolsai,2020,7.532803,6.918570,789
3,draft financial statements sustainability repo...,unipolsai,2020,6.647938,2.830496,1119
5,financial statements characterised reporting a...,unipolsai,2020,9.396564,0.848390,777
8,reporting period january december,unipolsai,2020,10.621443,6.493670,115
11,exceptions scope duly described relevant secti...,unipolsai,2020,10.341886,5.249770,454
...,...,...,...,...,...,...
57953,naples houses martyrdom saint last documented...,intesa,2017,9.534426,6.548452,794
57954,over naples via toledo naples via toledo rain ...,intesa,2017,9.531287,6.496984,794
57955,artist particularly renowned famous streets si...,intesa,2017,9.533686,6.537910,794
57956,picture part permanent collection display muse...,intesa,2017,9.526855,6.433837,794


In [107]:
fig = px.scatter(data, x="x", y="y", color="labels", hover_data=['text', 'name', 'year'])


In [108]:
data.to_csv('processed_data.csv')

In [137]:
d = {}
r = []
for i in data.labels.unique():
    if len(data[data.labels == i]) >200:
        r.append(i)
        text = ''
        minidf = data[data['labels']==i]
        for x in minidf.text:
            text += x
        kw_extractor = yake.KeywordExtractor(top=5, stopwords=None)
        keywords = kw_extractor.extract_keywords(text)
        for kw, v in keywords:
            d.update({i: (kw, v)})

df = pd.DataFrame.from_dict(d, orient='index', columns=['kw', 'score'])

In [140]:
df

,kw,score
777,pandemic emergency impact,1.291127e-05
995,offers insurance coverage,3.060903e-06
329,assessment suppliers supplier,1.253479e-06
571,stakeholder engagement activities,4.504091e-06
688,risk management function,2.987162e-07
900,hours training training,1.911589e-06
224,activitis targts rsults,1.422316e-06
753,data protection officer,4.726872e-07
1067,group tax ensures,2.477334e-06
268,trade union organisations,2.354847e-06


In [141]:
df = data[data['labels'].isin(r)].merge(df, right_on=df.index, left_on='labels')

In [142]:
df

,text,name,year,x,y,labels,kw,score
0,financial statements characterised reporting a...,unipolsai,2020,9.396564,0.848390,777,pandemic emergency impact,0.000013
1,based experience gained recent months document...,unipolsai,2020,9.435272,0.801850,777,pandemic emergency impact,0.000013
2,year significantly impacted consequences healt...,unipolsai,2020,9.592652,0.951381,777,pandemic emergency impact,0.000013
3,year marked indelibly health crisis group conf...,unipolsai,2020,9.485082,0.913787,777,pandemic emergency impact,0.000013
4,lastly please recall group since start health ...,unipolsai,2020,9.391704,0.667966,777,pandemic emergency impact,0.000013
...,...,...,...,...,...,...,...,...
5453,biodiversity degradation factor high relevance...,generali,2020,6.873899,2.621209,1109,areas key biodiversity,0.000008
5454,extinction many animal plant species impoveris...,generali,2020,4.455337,4.046981,1109,areas key biodiversity,0.000008
5455,natural ecosystems represents growing risk al...,generali,2020,4.450012,3.996513,1109,areas key biodiversity,0.000008
5456,reverse land degradation halt biodiversity bui...,unipolsai,2017,4.447273,4.034789,1109,areas key biodiversity,0.000008


In [113]:
df.to_csv('processed_datav1.csv')

In [116]:
from re import sub
import numpy as np
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex, SoftCosineSimilarity



In [147]:
def MatchSemantic(query_string, documents):
    l = []
    if len(documents) == 1: documents.append('')

    def preprocess(doc):
        # Tokenize, clean up input document string
       
        return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf"))]
    glove = api.load("glove-wiki-gigaword-50")
    similarity_index = WordEmbeddingSimilarityIndex(glove)

    # Preprocess the documents, including the query string
    corpus = [preprocess(document) for document in documents]
    for i in query_string:
        query = preprocess(i)

        # Load the model: this is a big file, can take a while to download and open
   
        # Build the term dictionary, TF-idf model
        dictionary = Dictionary(corpus + [query])
        tfidf = TfidfModel(dictionary=dictionary)

        # Create the term similarity matrix.
        similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

        query_tf = tfidf[dictionary.doc2bow(query)]

        index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)
        print(l)
        l.append(index[query_tf])
    return l


In [148]:
r

[777, 995, 329, 571, 688, 900, 224, 753, 1067, 268, 891, 1093, 794, 1060, 1109]

In [149]:
d1 = {}
terms = ['social', 'environment', 'governance']

for i in r:
        to_match = df[df['labels'] == i]
        to_match = list(to_match.text)
        arr = MatchSemantic(terms, to_match)
        d1.update({f'{i}': arr})

100%|██████████| 1321/1321 [00:15<00:00, 85.21it/s]
/Users/mattiafornasiero/opt/anaconda3/envs/tesi/lib/python3.10/site-packages/gensim/similarities/termsim.py:382: RuntimeWarning:

divide by zero encountered in divide

/Users/mattiafornasiero/opt/anaconda3/envs/tesi/lib/python3.10/site-packages/gensim/similarities/termsim.py:382: RuntimeWarning:

invalid value encountered in multiply

100%|██████████| 1900/1900 [00:21<00:00, 87.07it/s] 


In [156]:
for i in d1.keys():
    print(np.mean(d1[i]), i)

0.3292109 777
0.30568567 995
0.38785625 329
0.48186246 571
0.36420608 688
0.24979793 900
0.34036237 224
0.2933999 753
0.49405012 1067
0.2544428 268
0.37358788 891
0.33062252 1093
0.17313503 794
0.20854433 1060
0.32552603 1109


In [158]:
df[df['labels']==571]

,text,name,year,x,y,labels,kw,score
1371,regards relevance stakeholders order priority ...,unipolsai,2020,7.743648,2.397914,571,stakeholder engagement activities,0.000005
1372,materiality analysis also involvement stakehol...,unipolsai,2020,7.427099,2.532478,571,stakeholder engagement activities,0.000005
1373,accompanied constant commitment advocacy respo...,unipolsai,2020,8.008390,2.521442,571,stakeholder engagement activities,0.000005
1374,monitoring carried support external consultant,unipolsai,2020,8.522049,1.773387,571,stakeholder engagement activities,0.000005
1375,stakeholder relevance assessed stakeholder lis...,unipolsai,2021,7.722422,2.519065,571,stakeholder engagement activities,0.000005
...,...,...,...,...,...,...,...,...
1614,stakeholder stakeholder map represents banks n...,intesa,2017,7.635973,2.571232,571,stakeholder engagement activities,0.000005
1615,following table outlines features stakeholder ...,intesa,2017,7.863410,2.592685,571,stakeholder engagement activities,0.000005
1616,summary issues emerging stakeholder engagement...,intesa,2017,7.860708,2.577589,571,stakeholder engagement activities,0.000005
1617,company value solidity issue emerged listening...,intesa,2017,7.755096,2.527853,571,stakeholder engagement activities,0.000005


: 